In [1]:
import equinox as eqx
import gym
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import optax
import sys
import timeit
import wandb

from argparse import Namespace
from functools import partial
from jax import grad, jit, vmap
from typing import Sequence, Tuple

from jax_learning.agents.rl_agents import RLAgent
from jax_learning.buffers.ram_buffers import NextStateNumPyBuffer
from jax_learning.buffers.utils import batch_flatten, to_jnp
from jax_learning.constants import DISCRETE, CONTINUOUS
from jax_learning.rl_utils import interact, evaluate

from jax_learning.models import Policy, ActionValue, MLP, StochasticPolicy

In [2]:
wandb.init(project="test_jax_rl", group="reacher-reinforce_test")
wandb.define_metric("episodic_return", summary="max")

/Users/chanb/work/personal_research/jax_learning/.venv/lib/python3.10/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: chan. Use `wandb login --relogin` to force relogin
/Users/chanb/work/personal_research/jax_learning/.venv/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


In [3]:
cfg_dict = {
    # Environment setup
    "env": "Reacher-v2",
    "seed": 0,
    "render": False,
    "clip_action": True,
    "max_action": 1.,
    "min_action": -1.,
    
    # Experiment progress
    "load_step": 0,
    "log_interval": 5000,
    
    # Learning hyperparameters
    "gamma": 0.99,
    "max_timesteps": 1000000,
    "update_frequency": 128,
    "lr": 1e-2,
    "max_grad_norm": 10.,
    
    # Model architecture
    "hidden_dim": 256,
    "num_hidden": 2,
    
    # Evaluation
    "eval_cfg": {
        "max_episodes": 100,
        "seed": 1,
        "render": True,
        "clip_action": True,
        "max_action": 1.,
        "min_action": -1.,
    }
}
cfg = Namespace(**cfg_dict)
eval_cfg = Namespace(**cfg.eval_cfg)
wandb.config = cfg_dict

In [4]:
np.random.seed(cfg.seed)

In [5]:
env = gym.make(cfg.env)

/Users/chanb/work/personal_research/jax_learning/.venv/lib/python3.10/site-packages/gym/envs/registration.py:564: UserWarning: WARN: The environment Reacher-v2 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/Users/chanb/work/personal_research/jax_learning/.venv/lib/python3.10/site-packages/gym/envs/mujoco/mujoco_env.py:46: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
objc[29917]: Class GLFWWindowDelegate is implemented in both /usr/local/Cellar/glfw/3.3.7/lib/libglfw.3.3.dylib (0x1163be7b0) and /Users/chanb/.mujoco/mu

In [6]:
cfg.obs_dim = env.observation_space.shape
cfg.act_dim = env.action_space.shape
cfg.action_space = CONTINUOUS

In [7]:
cfg.h_state_dim = (1,)
cfg.rew_dim = (1,)

In [8]:
cfg

Namespace(env='Reacher-v2', seed=0, render=False, clip_action=True, max_action=1.0, min_action=-1.0, load_step=0, log_interval=5000, gamma=0.99, max_timesteps=1000000, update_frequency=128, lr=0.01, max_grad_norm=10.0, hidden_dim=256, num_hidden=2, eval_cfg={'max_episodes': 100, 'seed': 1, 'render': True, 'clip_action': True, 'max_action': 1.0, 'min_action': -1.0}, obs_dim=(11,), act_dim=(2,), action_space='continuous', h_state_dim=(1,), rew_dim=(1,))

In [9]:
cfg.buffer_rng = np.random.RandomState(cfg.seed)
cfg.env_rng = np.random.RandomState(cfg.seed)
cfg.agent_key, cfg.model_key = jrandom.split(jrandom.PRNGKey(cfg.seed), num=2)
eval_cfg.env_rng = np.random.RandomState(eval_cfg.seed)

In [15]:
from jax_learning.distributions import Distribution, Normal


class MLPGaussianPolicy(StochasticPolicy):
    obs_dim: int
    act_dim: int
    eps: float
    policy: eqx.Module

    def __init__(self,
                 obs_dim: Sequence[int],
                 act_dim: Sequence[int],
                 hidden_dim: int,
                 num_hidden: int,
                 key: jrandom.PRNGKey,
                 eps: float=1e-7):
        self.obs_dim = int(np.product(obs_dim))
        self.act_dim = int(np.product(act_dim))
        self.eps = eps
        self.policy = MLP(self.obs_dim, self.act_dim * 2, hidden_dim, num_hidden, key)

    def deterministic_action(self,
                             obs: np.ndarray,
                             h_state: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        act_mean, _ = jnp.split(self.policy(obs), 2, axis=-1)
        return act_mean, h_state
    
    def random_action(self,
                      obs: np.ndarray,
                      h_state: np.ndarray,
                      key: jrandom.PRNGKey) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        dist = self.dist(obs, h_state)
        act = dist.sample(key)
        return act, h_state
    
    def act_lprob(self,
                  obs: np.ndarray,
                  h_state: np.ndarray,
                  key: jrandom.PRNGKey) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        dist = self.dist(obs, h_state)
        act = dist.sample(key)
        lprob = dist.lprob(act)
        return act, lprob, h_state
    
    def dist(self,
             obs: np.ndarray,
             h_state: np.ndarray) -> Distribution:
        act_mean, act_raw_std = jnp.split(self.policy(obs), 2, axis=-1)
        act_std = jax.nn.softplus(act_raw_std) + self.eps
        return Normal(act_mean, act_std)


In [16]:
from jax_learning.buffers import ReplayBuffer
from jax_learning.learners import Learner

POLICY = "policy"
LOSS = "loss"
MEAN_LOSS = "mean_loss"
MAX_RETURN = "max_return"
MIN_RETURN = "min_return"
class REINFORCE(Learner):
    def __init__(self,
                 model: eqx.Module,
                 opt: optax.GradientTransformation,
                 buffer: ReplayBuffer,
                 cfg: Namespace):
        super().__init__(model, opt, buffer, cfg)
        
        self._step = cfg.load_step
        self._update_frequency = cfg.update_frequency
        self._sample_idxes = np.arange(cfg.update_frequency)
        self._gamma = cfg.gamma
        
        def get_lprob(dist, act):
            return jnp.sum(dist.lprob(act), keepdims=True)
        
        def score_function(lprob, ret):
            return lprob * ret
        
        @eqx.filter_grad(has_aux=True)
        def reinforce_loss(model: eqx.Module,
                           obss: np.ndarray,
                           h_states: np.ndarray,
                           acts: np.ndarray,
                           rets: np.ndarray) -> Tuple[np.ndarray, dict]:
            dists = jax.vmap(model.dist)(obss, h_states)
            lprobs = jax.vmap(get_lprob)(dists, acts)
            
            scores = jax.vmap(score_function)(lprobs, rets)
            loss = -jnp.mean(scores)
            return loss, {
                LOSS: loss,
                MAX_RETURN: jnp.max(rets),
                MIN_RETURN: jnp.min(rets),
                "dists": dists,
                "lprobs": lprobs,
                "scores": scores
            }
        
        def step(model: eqx.Module,
                 opt: optax.GradientTransformation,
                 opt_state: optax.OptState,
                 obss: np.ndarray,
                 h_states: np.ndarray,
                 acts: np.ndarray,
                 rets: np.ndarray) -> Tuple[eqx.Module, optax.OptState, jax.tree_util.PyTreeDef, dict]:
            grads, learn_info = reinforce_loss(model,
                                               obss,
                                               h_states,
                                               acts,
                                               rets)

            updates, opt_state = opt.update(grads, opt_state)
            model = eqx.apply_updates(model, updates)
            return model, opt_state, grads, learn_info
        self.step = eqx.filter_jit(step)
        
    def compute_returns(self, rews, dones):
        rets = np.zeros(rews.shape[0] + 1)
        for step in reversed(range(len(rews))):
            rets[step] = rets[step + 1] * self._gamma * (1 - dones[step]) + rews[step]
        return rets[:-1]

    def learn(self,
              next_obs: np.ndarray,
              next_h_state: np.ndarray,
              learn_info: dict):
        self._step += 1
        
        if self._step % self._update_frequency != 0:
            return

        obss, h_states, acts, rews, dones, _, _, _ = self.buffer.sample(batch_size=self._update_frequency,
                                                                        idxes=self._sample_idxes)

        rets = self.compute_returns(rews, dones)
        (obss, h_states, acts, rets) = to_jnp(*batch_flatten(obss,
                                                             h_states,
                                                             acts,
                                                             rets))
        model, opt_state, grads, curr_learn_info = self.step(model=self.model[POLICY],
                                                             opt=self.opt[POLICY],
                                                             opt_state=self.opt_state[POLICY],
                                                             obss=obss,
                                                             h_states=h_states,
                                                             acts=acts,
                                                             rets=rets)

        print(curr_learn_info["lprobs"])
        print(curr_learn_info["dists"].mean, curr_learn_info["dists"].std)
        print(curr_learn_info["scores"])
        print(model.policy.weights[-1].weight)
        print(model.policy.biases[-1])
        print(grads.policy.weights[-1].weight)
        print(grads.policy.biases[-1])
        print("=" * 100)
        assert self._step <= 6000

        self._model[POLICY] = model
        self._opt_state[POLICY] = opt_state

        learn_info[MEAN_LOSS] = curr_learn_info[LOSS].item()
        self.buffer.clear()

In [17]:
buffer = NextStateNumPyBuffer(
    buffer_size=cfg.update_frequency,
    obs_dim=cfg.obs_dim,
    h_state_dim=cfg.h_state_dim,
    act_dim=(1,) if cfg.action_space == DISCRETE else cfg.act_dim,
    rew_dim=cfg.rew_dim,
    rng=cfg.buffer_rng,
)

model = {
    POLICY: MLPGaussianPolicy(obs_dim=cfg.obs_dim,
                              act_dim=cfg.act_dim,
                              hidden_dim=cfg.hidden_dim,
                              num_hidden=cfg.num_hidden,
                              key=cfg.model_key,
                              eps=1e-4)
}

opt = {
    POLICY: optax.chain(optax.clip_by_global_norm(cfg.max_grad_norm),  # Clip by the gradient by the global norm
                        optax.scale_by_adam(),  # Use the updates from adam
                        optax.scale(-1.0)) # Gradient descent
}


learner = REINFORCE(model=model,
                    opt=opt,
                    buffer=buffer,
                    cfg=cfg)

agent = RLAgent(model=model,
                model_key=POLICY,
                buffer=buffer,
                learner=learner,
                key=cfg.agent_key)

In [18]:
%wandb

In [19]:
interact(env, agent, cfg)

[[-1.0910294]
 [-1.7010369]
 [-4.416682 ]
 [-1.6514733]
 [-1.1796126]
 [-6.382763 ]
 [-1.2415278]
 [-1.6817632]
 [-1.3658898]
 [-1.5978715]
 [-1.5620415]
 [-2.6773694]
 [-1.3686035]
 [-1.2207258]
 [-2.032523 ]
 [-2.9138284]
 [-1.3762307]
 [-1.4547474]
 [-1.5658554]
 [-2.752273 ]
 [-2.3574128]
 [-1.7982812]
 [-1.1981305]
 [-2.5892181]
 [-2.5536783]
 [-2.8135762]
 [-2.036829 ]
 [-1.3046618]
 [-1.57158  ]
 [-2.049324 ]
 [-1.2603221]
 [-3.6423445]
 [-2.437175 ]
 [-1.5148313]
 [-1.4176459]
 [-3.1579876]
 [-1.7641993]
 [-1.7174438]
 [-1.3036623]
 [-1.8136446]
 [-2.1237602]
 [-2.0036387]
 [-1.9474576]
 [-1.4955722]
 [-1.876626 ]
 [-2.9864473]
 [-4.0532703]
 [-1.609286 ]
 [-1.2316461]
 [-2.1705184]
 [-2.0403814]
 [-2.039529 ]
 [-1.254119 ]
 [-1.7612857]
 [-1.7910677]
 [-4.123208 ]
 [-1.3277681]
 [-1.137247 ]
 [-1.3364236]
 [-0.8767302]
 [-1.3515855]
 [-1.2727724]
 [-1.3137594]
 [-0.9980736]
 [-1.9036696]
 [-3.2515106]
 [-0.5569221]
 [-4.9689007]
 [-1.658597 ]
 [-3.0550785]
 [-3.4758193]
 [-2.4

[[-26.778578]
 [-28.354488]
 [-27.74334 ]
 [-25.981659]
 [-25.90392 ]
 [-25.79721 ]
 [-24.913763]
 [-25.575325]
 [-24.201351]
 [-26.12061 ]
 [-26.079157]
 [-25.22062 ]
 [-25.868538]
 [-22.0043  ]
 [-23.115192]
 [-22.313526]
 [-20.921158]
 [-21.680967]
 [-24.572428]
 [-23.688047]
 [-24.298029]
 [-25.37779 ]
 [-21.491516]
 [-22.649565]
 [-24.69767 ]
 [-24.702682]
 [-25.094784]
 [-26.557156]
 [-27.536736]
 [-26.115292]
 [-26.279808]
 [-26.68435 ]
 [-26.59713 ]
 [-26.047405]
 [-27.137245]
 [-26.27406 ]
 [-26.297022]
 [-27.539482]
 [-26.480133]
 [-30.635113]
 [-30.495567]
 [-27.687462]
 [-27.550558]
 [-30.184578]
 [-27.149826]
 [-27.13456 ]
 [-28.071903]
 [-27.61156 ]
 [-28.205647]
 [-27.528145]
 [-27.50887 ]
 [-28.305876]
 [-28.246029]
 [-28.622803]
 [-29.884695]
 [-27.838507]
 [-28.169771]
 [-28.267536]
 [-28.458324]
 [-28.46266 ]
 [-28.915833]
 [-27.773945]
 [-30.744709]
 [-28.172451]
 [-28.086082]
 [-27.96851 ]
 [-27.963764]
 [-28.576046]
 [-28.479427]
 [-28.113194]
 [-30.168663]
 [-28.

[[-1.9532041e+05]
 [-7.8125800e+05]
 [-9.9318686e+00]
 [-3.1250095e+06]
 [-7.8125756e+05]
 [-6.6412859e+00]
 [-5.7116365e+00]
 [-4.8836637e+04]
 [-1.9531909e+05]
 [-6.6609850e+00]
 [-1.9531861e+05]
 [-6.6419115e+00]
 [-4.8834438e+04]
 [-6.3269157e+00]
 [-7.8125694e+05]
 [-6.5891771e+00]
 [-7.8125819e+05]
 [-1.9531930e+05]
 [-7.0131559e+00]
 [-7.8125794e+05]
 [-7.8125688e+05]
 [-7.8125700e+05]
 [-7.2064695e+00]
 [-3.1250075e+06]
 [-3.1250075e+06]
 [-7.8125988e+05]
 [-7.0699520e+00]
 [-7.8879976e+00]
 [-3.1250080e+06]
 [-8.4732656e+00]
 [-7.1649637e+00]
 [-7.8125725e+05]
 [-7.7152605e+00]
 [-7.8125944e+05]
 [-3.1250082e+06]
 [-8.3010492e+00]
 [-3.1250078e+06]
 [-3.1250075e+06]
 [-7.8125738e+05]
 [-3.1250082e+06]
 [-3.1250078e+06]
 [-7.5033789e+00]
 [-7.8125769e+05]
 [-3.1250078e+06]
 [-2.7339830e+00]
 [-5.9153585e+00]
 [-1.2212923e+04]
 [-7.4648161e+00]
 [-6.3795948e+00]
 [-1.9531914e+05]
 [-6.8121166e+00]
 [-6.7009468e+00]
 [-1.9531970e+05]
 [-1.9532028e+05]
 [-2.0281235e+01]
 [-3.05600

[[-1.00189314e+01]
 [-7.97774601e+00]
 [-1.00297804e+01]
 [-1.25000100e+07]
 [-8.31366062e+00]
 [-1.25000110e+07]
 [-8.34286022e+00]
 [-1.25000100e+07]
 [-9.43476009e+00]
 [-1.25000100e+07]
 [-5.00000120e+07]
 [-1.25000100e+07]
 [-1.25000100e+07]
 [-8.37762928e+00]
 [-8.43308353e+00]
 [-8.53144741e+00]
 [-1.94801758e+02]
 [-7.66459412e+02]
 [-7.81256625e+05]
 [-7.81257375e+05]
 [-3.12500775e+06]
 [-1.25000090e+07]
 [-8.82810688e+00]
 [-8.11950207e+00]
 [-9.95440769e+00]
 [-1.25000100e+07]
 [-3.12500850e+06]
 [-7.70593643e+00]
 [-3.12500825e+06]
 [-7.94602108e+00]
 [-8.64083767e+00]
 [-5.00000120e+07]
 [-1.25000100e+07]
 [-1.25000120e+07]
 [-5.00000120e+07]
 [-8.32802486e+00]
 [-8.42088604e+00]
 [-8.75892735e+00]
 [-9.17591953e+00]
 [-8.32697201e+00]
 [-1.02129107e+01]
 [-9.41854572e+00]
 [-1.25000100e+07]
 [-8.40147114e+00]
 [-1.25000100e+07]
 [-1.25000100e+07]
 [-1.25000100e+07]
 [-5.00000120e+07]
 [-8.45213604e+00]
 [-8.58664799e+00]
 [-1.12500016e+08]
 [-1.25000100e+07]
 [-1.2500014

[[-7.0557919e+00]
 [-1.2500009e+07]
 [-1.2500011e+07]
 [-7.8859301e+00]
 [-8.0991812e+00]
 [-8.1090841e+00]
 [-1.2500010e+07]
 [-1.2500010e+07]
 [-9.0867491e+00]
 [-1.2500010e+07]
 [-5.0000012e+07]
 [-2.0000002e+08]
 [-8.9735746e+00]
 [-8.8900900e+00]
 [-2.0000002e+08]
 [-5.0000012e+07]
 [-8.8706255e+00]
 [-8.9257383e+00]
 [-9.2189360e+00]
 [-5.0000012e+07]
 [-9.0589342e+00]
 [-5.0000012e+07]
 [-1.0525529e+01]
 [-9.1165247e+00]
 [-2.0000002e+08]
 [-9.6362391e+00]
 [-9.2525225e+00]
 [-2.0000002e+08]
 [-1.0206313e+01]
 [-5.0000012e+07]
 [-9.4426889e+00]
 [-9.3053446e+00]
 [-9.3557291e+00]
 [-2.0000002e+08]
 [-2.0000002e+08]
 [-2.0000002e+08]
 [-2.0000002e+08]
 [-2.0000002e+08]
 [-3.0653009e+00]
 [-7.8125831e+05]
 [-1.2500009e+07]
 [-1.2500010e+07]
 [-1.2500010e+07]
 [-9.7652082e+00]
 [-5.0000012e+07]
 [-8.6796751e+00]
 [-5.0000012e+07]
 [-5.0000016e+07]
 [-8.1534929e+00]
 [-8.7563772e+00]
 [-1.1250002e+08]
 [-5.0000012e+07]
 [-8.7284040e+00]
 [-5.0000012e+07]
 [-5.0000016e+07]
 [-1.00857

...

[[-8.00000064e+08]
 [-9.93802929e+00]
 [-1.03205214e+01]
 [-1.12622118e+01]
 [-3.20000026e+09]
 [-3.20000026e+09]
 [-1.14890280e+01]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-3.20000026e+09]
 [-1.28000010e+10]
 [-1.03435831e+01]
 [-1.07507944e+01]
 [-3.20000026e+09]
 [-1.28000010e+10]
 [-1.35087500e+01]
 [-1.28000010e+10]
 [-1.12257166e+01]
 [-1.40525084e+01]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-1.19923735e+01]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-1.27512064e+01]
 [-1.22846842e+01]
 [-1.28000010e+10]
 [-3.20000026e+09]
 [-3.20000026e+09]
 [-1.16886511e+01]
 [-1.28000010e+10]
 [-3.20000026e+09]
 [-3.20000026e+09]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-1.14879408e+01]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.25354471e+01]
 [-1.28000010e+10]
 [-1.15200008e+11]
 [-1.22090120e+01]
 [-1.28000010e+10]
 [-1.18847799e+01]
 [-1.13769655e+01]
 [-4.88336172e+04]
 [-5.00000120e+07]
 [-2.00000016e+08]
 [-3.20000026e+09]
 [-2.00000016e+08]
 [-8.13033962e+00]
 [-9.8554029

[[-1.28000010e+10]
 [-5.12000041e+10]
 [-1.30190248e+01]
 [-1.26918173e+01]
 [-1.28000010e+10]
 [-1.26498919e+01]
 [-5.12000041e+10]
 [-1.20439043e+01]
 [-1.21206656e+01]
 [-1.22285395e+01]
 [-1.23226271e+01]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.22439451e+01]
 [-5.12000041e+10]
 [-1.30647573e+01]
 [-5.12000041e+10]
 [-1.33768187e+01]
 [-5.47911835e+00]
 [-7.03125800e+06]
 [-7.81256875e+05]
 [-8.42310238e+00]
 [-3.12500800e+06]
 [-1.25000090e+07]
 [-1.05423651e+01]
 [-5.00000120e+07]
 [-8.63658810e+00]
 [-2.00000016e+08]
 [-3.20000026e+09]
 [-1.26686468e+01]
 [-1.18975325e+01]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-1.16772261e+01]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-1.18159971e+01]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-1.20349112e+01]
 [-5.12000041e+10]
 [-1.28000010e+10]
 [-1.19178057e+01]
 [-5.12000041e+10]
 [-1.2608183

[[-1.28000010e+10]
 [-1.32158327e+01]
 [-1.28000010e+10]
 [-1.19385061e+01]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.22515917e+01]
 [-1.25248976e+01]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.43451586e+01]
 [-5.12000041e+10]
 [-1.22627249e+01]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.24362020e+01]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-2.04800016e+11]
 [-1.20666952e+01]
 [-1.21832514e+01]
 [-5.12000041e+10]
 [-1.23642530e+01]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.24885759e+01]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-1.24451818e+01]
 [-2.04800016e+11]
 [-1.27567225e+01]
 [-5.12000041e+10]
 [-1.28275118e+01]
 [-1.26914682e+01]
 [-2.04800016e+11]
 [-1.30344076e+01]
 [-5.33221817e+00]
 [-9.20905399e+00]
 [-2.00000016e+08]
 [-1.05262270e+01]
 [-9.20824718e+00]
 [-8.00000064e+08]
 [-3.20000026e+09]
 [-1.07557344e+01]
 [-1.08792791e+01]
 [-1.19886637e+01]
 [-1.28000010e+10]
 [-3.20000026e+09]
 [-1.2800001

[[-1.24986620e+01]
 [-2.04800016e+11]
 [-2.04800016e+11]
 [-2.04800016e+11]
 [-5.12000041e+10]
 [-1.27779570e+01]
 [-2.04800016e+11]
 [-1.36112146e+01]
 [-5.12000041e+10]
 [-5.12000041e+10]
 [-2.04800016e+11]
 [-1.27150087e+01]
 [-4.88338945e+04]
 [-7.81258000e+05]
 [-7.81256500e+05]
 [-7.81362057e+00]
 [-5.00000160e+07]
 [-3.12500800e+06]
 [-8.89557362e+00]
 [-1.23544607e+01]
 [-8.01467991e+00]
 [-1.25000090e+07]
 [-3.20000026e+09]
 [-1.08248205e+01]
 [-1.15131254e+01]
 [-1.17392168e+01]
 [-1.09236383e+01]
 [-1.21774302e+01]
 [-1.08928270e+01]
 [-1.39988585e+01]
 [-1.18608103e+01]
 [-1.11282988e+01]
 [-3.20000026e+09]
 [-1.28000010e+10]
 [-1.28000010e+10]
 [-1.26525946e+01]
 [-5.12000041e+10]
 [-3.20000026e+09]
 [-1.28000010e+10]
 [-3.20000026e+09]
 [-1.17052164e+01]
 [-1.14997263e+01]
 [-1.28000010e+10]
 [-1.29045286e+01]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-1.19429388e+01]
 [-5.12000041e+10]
 [-1.28000010e+10]
 [-5.12000041e+10]
 [-1.21228838e+01]
 [-5.12000041e+10]
 [-5.1200004

AssertionError: 

In [ ]:
evaluate(env, agent, eval_cfg)